In [9]:
#import libraries and setup sql cursor
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import matplotlib.pyplot as plt
import seaborn as sb

conn = sqlite3.connect('data/movies.db')
cur = conn.cursor()

## Get bom and rotten tomatoe data and merge them into one with overlapping values(inner)

In [10]:
#get rotten tomatoe data
rt_movies = pd.DataFrame(cur.execute("SELECT * FROM rotten_tomatoes_movies").fetchall())
rt_movies.columns = [i[0] for i in cur.description]

OperationalError: no such table: rotten_tomatoes_movies

In [11]:
#get bom movie data
bom_movie_gross = pd.DataFrame(cur.execute("SELECT foreign_gross, domestic_gross, title from bom_movie_gross").fetchall())
bom_movie_gross.columns = [i[0] for i in cur.description]
bom_movie_gross = bom_movie_gross[bom_movie_gross["foreign_gross"].notna()]

OperationalError: no such table: bom_movie_gross

In [ ]:
#create merged dataframe of rotten tomatoe and bom movies
bom_gross_rt_genre = rt_movies.join(bom_movie_gross, how="inner")

In [ ]:
#drop na values from rotten tomatoe and bom movie merge
bom_gross_rt_genre.dropna(inplace=True)

In [ ]:
not_list_format =  bom_gross_rt_genre

In [ ]:
#sc
not_list_format.head()

## get list of unique genres

# Get data based off a certain genre from bom and rotten tomatoe merge

In [ ]:
#function makes new databse with just a certain column
def get_genre_db(db, genre):
    return db[db["genres"].str.contains(genre)]

In [ ]:
action = get_genre_db(bom_gross_rt_genre, "Action")

In [ ]:
#reformat genres data go make it a list of items
def get_unique_genres(raw_genres):
    all_genres = []
    genres = raw_genres.str.replace("&", ",")
    genres = raw_genres.str.split(',')
    for item in genres:
        if item != None:
            for genre in item:
                all_genres.append(genre)
    unique_genres = set(all_genres)
    unique_genres = set([(genre.strip()) for genre in unique_genres])
    return unique_genres
get_unique_genres(bom_gross_rt_genre["genres"])

In [ ]:
def get_unique_content_ratings(raw_content_ratings):
    all_content_ratings = []
    for rating in raw_content_ratings:
        all_content_ratings.append(rating)
    unique_content_ratings = set(all_content_ratings)
    return unique_content_ratings
unique_content_ratings = get_unique_content_ratings(action["content_rating"])

In [ ]:
def get_unique_people(raw_people):
    all_people = []
    clumped_people = []
    clumped_people = raw_people.str.split(",")
    for row in clumped_people:
        if row != None:
            for director in row:
                all_people.append(director)
    unique_people = set(all_people)
    unique_people = [(director.strip()) for director in unique_people]
    unique_people = set(unique_people)
    return unique_people

In [ ]:
unique_directors = get_unique_people(action["directors"])

In [ ]:
action[action["actors"].str.contains("Queen")]["actors"]

### gross by audience rating

In [ ]:
action.sort_values("audience_rating", ascending=False, inplace=True)

In [ ]:
gross_rating = sb.scatterplot(action["domestic_gross"], action["audience_rating"])
gross_rating.set(ylabel ="Audience rating(/100)", xlabel = "Domestic Gross(*100mil)", title ='Audience rating compared to Domestic Gross')

In [ ]:
#sc
action[action["directors"].str.contains("Tim Kirkby")]

In [ ]:
top_gross_rating = sb.scatterplot(action["domestic_gross"][0:10], action["audience_rating"][0:10])
top_gross_rating.set(ylabel ="Audience rating(/100)", xlabel = "Domestic Gross(*100mil)", title ='some title')

### content rating by audience rating

In [ ]:
cr_ar = action.groupby(['content_rating'])['audience_rating'].mean().sort_values(ascending=False)

In [ ]:
cr_ar

In [ ]:
cr_ar_sb = sb.barplot(list(cr_ar.keys()), cr_ar)
cr_ar_sb.set(ylabel ="Audience rating of 100", xlabel = "Content Rating", title ='Audience rating based on content rating')

### top audience rating means by director

In [ ]:
action.groupby(['directors'])['audience_rating'].mean().sort_values(ascending=False)

In [ ]:
#sc
len(action)

In [ ]:
def groupby_unformatted_column(df, unique, unique_category, aggregate_category):
    #unique is a list of unique values of the unique category
    return_list = []
    occurances_list = []
    label_list = []
    return_dict = {}
    for item in unique:
        frame = df[df[unique_category].str.contains(item)]
        aggregate = round(frame[aggregate_category].mean(), 1)
        occurance = len(frame[aggregate_category])
        occurances_list.append(occurance)
        return_list.append(aggregate)
    return_dict = {unique_category: list(unique), aggregate_category: return_list, "occurances": occurances_list}
    df = pd.DataFrame(return_dict)
    df["labels"] = df[unique_category].astype(str) + " (" + df["occurances"].astype(str) + ")"
    df.sort_values(aggregate_category, inplace=True, ascending = False)
    return df


In [ ]:
rating_director_df = groupby_unformatted_column(action, unique_directors, "directors", "audience_rating")

In [ ]:
unique_directors[35]

In [ ]:
rating_director_df.drop_duplicates(subset="directors", inplace=True)

In [ ]:
rating_director_df

In [ ]:
#can be used for labels
top_directors_by_rating = rating_director_df[rating_director_df["occurances"]> 2]

In [ ]:
#sc
top_directors_by_rating

In [ ]:
tdbar = sb.barplot(top_directors_by_rating["audience_rating"], top_directors_by_rating["directors"], orient = 'h')
tdbar.set(xlabel="Audience Rating out of 100", title="Mean Audience Rating of movies by directors(top)")
tdbar.set_yticklabels(top_directors_by_rating["labels"])

### top audience rating by actor

In [ ]:
unique_actors = get_unique_people(action["actors"])

In [ ]:
unique_actors

In [ ]:
rating_by_actors = groupby_unformatted_column(action, unique_actors, "actors", "audience_rating")

In [ ]:
rating_by_actors = rating_by_actors[rating_by_actors["occurances"]>0][0:15]

In [ ]:
rating_by_actors.head(3)

In [ ]:
sb.barplot(rating_by_actors["audience_rating"], rating_by_actors["labels"])

### Top Grossing means by director

#### top directors by mean Foreign Gross

In [ ]:
fgross_by_director = groupby_unformatted_column(action, unique_directors, "actors", "foreign_gross")

In [ ]:
fgross_director_df[0:10]

In [ ]:
#sc
top_directors_by_fgross

In [ ]:
tdbfg = sb.barplot(top_directors_by_fgross["foreign_gross"], top_directors_by_fgross["director"], orient = 'h')
tdbfg.set(xlabel = "Foreign gross(*100mil)", title="Mean Foreign Gross of Movies by director(top)")
tdbfg.set_yticklabels(top_directors_by_fgross["labels"])

In [ ]:
dgross_director_df = groupby_unformatted_column(action, unique_directors, "directors", "domestic_gross")

In [ ]:
top_directors_by_dgross = dgross_director_df[0:10]

In [ ]:
#sc
top_directors_by_dgross

#### top directors by mean Gross domestic 

In [ ]:
tdbdg = sb.barplot(top_directors_by_dgross["domestic_gross"], top_directors_by_dgross["directors"], orient = 'h')
tdbdg.set(xlabel = "Domestic gross(*100mil)", title="Mean Domestic Gross of Movies by director(top)")
tdbdg.set_yticklabels(top_directors_by_dgross["labels"])

### top directors by domestic gross/audience rating

In [ ]:
#add new column to action that divides dgross by audience rating
action["dgross_and_rating"] = action["domestic_gross"]/ action["audience_rating"]

In [ ]:
dgross_rating_director_df = groupby_unformatted_column(action, unique_directors, "directors", "dgross_and_rating")

In [ ]:
top_dgrd = dgross_rating_director_df[0:10]

In [ ]:
tdbdgar = sb.barplot(top_dgrd["dgross_and_rating"], top_dgrd["directors"], orient = 'h')
tdbdgar.set(xlabel = "Domestic gross/audience rating", title="Directors with Highest Mean Domestic Gross / Audience Rating Scpre")
tdbdgar.set_yticklabels(top_dgrd["labels"])

In [ ]:
#turn string values within foreign gross column into na and drop them
not_list_format["foreign_gross"] = not_list_format["foreign_gross"].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

### Genre by domestic gross Revenue

In [ ]:
genre_dgross_df = groupby_unformatted_column(not_list_format, unique_genres, "genres", "domestic_gross")

In [ ]:
genre_dgross_df

In [ ]:
genre_dgross_df = genre_dgross_df[genre_dgross_df["genres"] != 'Cult Movies']
genre_dgross_df = genre_dgross_df[genre_dgross_df["genres"] != 'Television']
genre_dgross_df = genre_dgross_df[genre_dgross_df["genres"] != 'Classics']
genre_dgross_df = genre_dgross_df[genre_dgross_df["genres"] != 'Art House & International']

In [ ]:
genre_dgross_df.sort_values("domestic_gross", ascending=False, inplace=True)

In [ ]:
genre_dgross_df

In [ ]:
#make graph of mean gross by genre
fig, ax = plt.subplots(figsize= (15, 8))
ax.barh(genre_dgross_df["genres"], genre_dgross_df["domestic_gross"])
ax.set_title("Domestic Gross Revenue Average by Genre")
ax.set_yticklabels(genre_dgross_df["labels"])
ax.set_xlabel("Domestic Gross(x10 Million)",fontsize=15)
ax.set_ylabel("Genre",fontsize=15)

### Genre by audience rating

In [ ]:
genre_rating_df = groupby_unformatted_column(not_list_format, unique_genres, "genres", "audience_rating")

In [ ]:
gbar = sb.barplot(genre_rating_df["audience_rating"], genre_rating_df["genres"], orient="h")
gbar.set_yticklabels(genre_rating_df["labels"])